# 前準備

前準備を始める前に、MarletplaceでCohere Rerank3のサブスクリプションを行ってください

https://aws.amazon.com/marketplace/pp/prodview-ydysc72qticsw?sr=0-2&ref_=beagle&applicationId=AWSMPContessa

CDKを使って以下の環境を構築します。

* OpenSearch Service
* SageMaker
* Lambda
* その他IAMなどのリソース

OpenSearchから外部モデルを使ったRAGを行う場合、動作サポートされているモデルが、OpenAIのモデルかAnthropic Claude 2のみです。

今回は、Mistral Large 2を使用したいのですが、直接指定するとうまくいかなかったので、Lambda上でLiteLLMを動作させ、OpenAIのインターフェイスでBedrockにアクセスさせる構成としました。


参考
https://github.com/cohere-ai/cohere-aws/blob/main/notebooks/sagemaker/rerank_v3_notebooks/Deploy%20rerank%20multilingual%20v3.0%20model.ipynb


## CDKのデプロイ

CDKでOpenSearchやSageMakerをデプロイします

MASTER_USER_ARNにはこのノートブックを実行している環境のIAMユーザーやロールのARNをセットします

In [4]:
%%bash

MASTER_USER_ARN=arn:aws:iam::123456789012:role/**********

npm install -g aws-cdk
cd opensearch-cdk/
npm i

cdk bootstrap

cdk deploy \
    --parameters OpensearchCdkStack:masterUserArn=${MASTER_USER_ARN} \
    --parameters OpensearchCdkStack:deployRerankModel=true \
    --all \
    --require-approval never


changed 1 package in 2s


npm warn deprecated inflight@1.0.6: This module is not supported, and leaks memory. Do not use it. Check out lru-cache if you want a good and tested way to coalesce async requests by a key value, which is much more comprehensive and powerful.
npm warn deprecated glob@7.2.3: Glob versions prior to v9 are no longer supported



added 311 packages, and audited 346 packages in 11s

34 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities


 ⏳  Bootstrapping environment aws://637423213562/us-east-1...
 ⏳  Bootstrapping environment aws://637423213562/us-west-2...


Trusted accounts for deployment: (none)
Trusted accounts for lookup: (none)


...

Trusted accounts for deployment: (none)
Trusted accounts for lookup: (none)


Using default execution policy of 'arn:aws:iam::aws:policy/AdministratorAccess'. Pass '--cloudformation-execution-policies' to customize.
 ✅  Environment aws://637423213562/us-west-2 bootstrapped (no changes).
 ✅  Environment aws://637423213562/us-east-1 bootstrapped (no changes).

#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 1.32kB done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/sam/build-python3.12:latest
#2 DONE 0.9s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/2] FROM public.ecr.aws/sam/build-python3.12:latest@sha256:c568dbcb27c908c9d24fc039e5190a1e185a34691d7cec7910db86a3a993f9c5
#4 DONE 0.0s

#5 [2/2] RUN     python -m venv /usr/app/venv &&     mkdir /tmp/pip-cache &&     chmod -R 777 /tmp/pip-cache &&     pip install --upgrade pip &&     mkdir /tmp/poetry-cache &&     chmod -R 777 /tmp/poetry-cache &&     pip install pipenv==2022

arn:aws:cloudformation:us-east-1:637423213562:stack/OpensearchCdkStack/3810f5f0-5a94-11ef-9ac0-12cca92520cf


LiteLLMLambdaCdkStack: creating CloudFormation changeset...
LiteLLMLambdaCdkStack | 0/7 | 11:42:08 PM | REVIEW_IN_PROGRESS   | AWS::CloudFormation::Stack | LiteLLMLambdaCdkStack User Initiated
LiteLLMLambdaCdkStack | 0/7 | 11:42:15 PM | CREATE_IN_PROGRESS   | AWS::CloudFormation::Stack | LiteLLMLambdaCdkStack User Initiated
LiteLLMLambdaCdkStack | 0/7 | 11:42:18 PM | CREATE_IN_PROGRESS   | AWS::Lambda::LayerVersion | LiteLLMLayer (LiteLLMLayer9F8C78FF) 
LiteLLMLambdaCdkStack | 0/7 | 11:42:18 PM | CREATE_IN_PROGRESS   | AWS::IAM::Role            | litellmFunction/ServiceRole (litellmFunctionServiceRole7133C1AF) 
LiteLLMLambdaCdkStack | 0/7 | 11:42:18 PM | CREATE_IN_PROGRESS   | AWS::CDK::Metadata        | CDKMetadata/Default (CDKMetadata) 
LiteLLMLambdaCdkStack | 0/7 | 11:42:19 PM | CREATE_IN_PROGRESS   | AWS::CDK::Metadata        | CDKMetadata/Default (CDKMetadata) Resource creation Initiated
LiteLLMLambdaCdkStack | 1/7 | 11:42:19 PM | CREATE_COMPLETE      | AWS::CDK::Metadata        |

arn:aws:cloudformation:us-west-2:637423213562:stack/LiteLLMLambdaCdkStack/d1993e10-5a96-11ef-a9de-02a8cfaa94f3


CDKのOutputから値をセットします

In [8]:
oss_region = "us-east-1"

# OpensearchCdkStack.ossdomainendpoint
oss_domain_endpoint = "search-domain66ac69e0-vbssjyumvvio-oi6ssvzf3rl7ea23ppmic4tpuq.us-east-1.es.amazonaws.com"
# OpensearchCdkStack.ossdomainname
oss_domain_name = "domain66ac69e0-vbssjyumvvio"
# OpensearchCdkStack.ossexecutionrole
oss_execution_role = "arn:aws:iam::637423213562:role/OpensearchCdkStack-roleforopensearch6A669598-r9SRkLZVEQ8G"
# OpensearchCdkStack.osssecretarn
oss_secret_arn = "arn:aws:secretsmanager:us-east-1:637423213562:secret:openapidummykeyF27E0F80-PZnvu7zb1iRz-ZTETdn"
# OpensearchCdkStack.sagemakerrerankendpoint
sagemaker_rerank_endpoint = "https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/Rerank3Endpoint/invocations"

# LiteLLMLambdaCdkStack.litellmlambdaendpoint
litellm_lambda_endpoint = "https://oogkxboy2rtd5olvzqe26vfhdu0uaoov.lambda-url.us-west-2.on.aws/"

---

# OpenSearchの環境構築

In [5]:
import boto3
import nest_asyncio
import pandas as pd
from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv(override=True)

True

## Sudachiプラグインの導入

Sudachiプラグインは、OpenSearchの日本語解析プラグインです。


「G38656169」は、OpenSearch 2.13用のSudachiプラグインのパッケージIDです。


In [9]:
def associate_package(domain_name: str, package_id: str = "G38656169"):
    oss_client = boto3.client("opensearch")

    return oss_client.associate_package(PackageID=package_id, DomainName=domain_name)


associate_package(domain_name=oss_domain_name)

{'ResponseMetadata': {'RequestId': '519e0bc8-1f6d-4e53-a5ed-5f5190b2072c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '519e0bc8-1f6d-4e53-a5ed-5f5190b2072c',
   'content-type': 'application/json',
   'content-length': '283',
   'date': 'Wed, 14 Aug 2024 23:45:16 GMT'},
  'RetryAttempts': 0},
 'DomainPackageDetails': {'PackageID': 'G38656169',
  'PackageName': 'analysis-sudachi',
  'PackageType': 'ZIP-PLUGIN',
  'LastUpdated': datetime.datetime(2024, 8, 14, 23, 45, 17, 41000, tzinfo=tzlocal()),
  'DomainName': 'domain66ac69e0-vbssjyumvvio',
  'DomainPackageStatus': 'ASSOCIATING',
  'PackageVersion': 'v1'}}

Sudachiプラグインの適用完了を待機します。30分ほどかかります

In [10]:
def list_packages_for_domain(domain_name: str, package_id: str = "G38656169"):
    import boto3

    oss_client = boto3.client("opensearch")

    response = oss_client.list_packages_for_domain(DomainName=domain_name)

    for tmp in response["DomainPackageDetailsList"]:

        name = tmp["DomainName"]
        id = tmp["PackageID"]
        status = tmp["DomainPackageStatus"]

        if name == domain_name and id == package_id:
            return status


while list_packages_for_domain(domain_name=oss_domain_name) in ["ASSOCIATING"]:
    print(".", end="")

    import time

    time.sleep(10)


print("適用完了")

...............................................................................................................................................適用完了


---

## OpenSearchクライアントを作成

In [11]:
from urllib.parse import urlparse

from boto3 import Session
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

url = urlparse(f"https://{oss_domain_endpoint}")
service = "es"

credentials = Session().get_credentials()

auth = AWSV4SignerAuth(credentials, oss_region, service)

client = OpenSearch(
    hosts=[{"host": url.netloc, "port": url.port or 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=120,
)

info = client.info()
print(f"{info['version']['distribution']}: {info['version']['number']}")

opensearch: 2.13.0


## 外部AIモデルの登録

作成済みモデルのIDを取得する関数

In [12]:
def get_model_id(model_name: str):

    body = {
        "query": {"match": {"name": model_name}},
        "sort": [{"last_updated_time": {"order": "desc"}}],
        "size": 1,
    }

    response = client.transport.perform_request(
        method="POST", url="/_plugins/_ml/models/_search", body=body
    )

    return response["hits"]["hits"][0]["_id"]

### Bedrock の Titan Text Embeddings V2

In [13]:
titan_embed_model = "amazon.titan-embed-text-v2:0"
titan_embed_region = "us-east-1"

body = {
    "name": "Amazon Bedrock Connector: Titan Text Embeddings V2",
    "description": "The connector to the Bedrock Titan Text Embeddings V2",
    "version": 1,
    "protocol": "aws_sigv4",
    "parameters": {"region": titan_embed_region, "service_name": "bedrock"},
    "credential": {"roleArn": oss_execution_role},
    "actions": [
        {
            "action_type": "predict",
            "method": "POST",
            "url": f"https://bedrock-runtime.{titan_embed_region}.amazonaws.com/model/{titan_embed_model}/invoke",
            "headers": {
                "content-type": "application/json",
                "x-amz-content-sha256": "required",
            },
            "request_body": '{ "inputText": "${parameters.inputText}" }',
            "pre_process_function": '\n    StringBuilder builder = new StringBuilder();\n    builder.append("\\"");\n    String first = params.text_docs[0];\n    builder.append(first);\n    builder.append("\\"");\n    def parameters = "{" +"\\"inputText\\":" + builder + "}";\n    return  "{" +"\\"parameters\\":" + parameters + "}";',
            "post_process_function": '\n      def name = "sentence_embedding";\n      def dataType = "FLOAT32";\n      if (params.embedding == null || params.embedding.length == 0) {\n        return params.message;\n      }\n      def shape = [params.embedding.length];\n      def json = "{" +\n                 "\\"name\\":\\"" + name + "\\"," +\n                 "\\"data_type\\":\\"" + dataType + "\\"," +\n                 "\\"shape\\":" + shape + "," +\n                 "\\"data\\":" + params.embedding +\n                 "}";\n      return json;\n    ',
        }
    ],
}

response = client.transport.perform_request(
    method="POST", url="/_plugins/_ml/connectors/_create", body=body
)

titan_embed_connector_id = response["connector_id"]


body = {
    "name": "titan-embedded-model",
    "function_name": "remote",
    "connector_id": titan_embed_connector_id,
}

response = client.transport.perform_request(
    method="POST", url="/_plugins/_ml/models/_register", body=body
)

titan_embed_model_id = response["model_id"]
titan_embed_model_id

'MyJgU5EB_plnzCpaLhTK'

作成済みのモデルIDを取得する場合

In [14]:
titan_embed_model_id = get_model_id("titan-embedded-model")
titan_embed_model_id

'MyJgU5EB_plnzCpaLhTK'

### SageMaker の Rerank3

In [38]:
coehre_rerank3_region = "us-east-1"
coehre_rerank3_model = "rerank-multilingual-v3.0"
coehre_rerank3_endpoint = sagemaker_rerank_endpoint

body = {
    "name": "Amazon SageMaker Connector: Cohere Rerank 3 Model - Multilingual",
    "description": "The connector to the Cohere Rerank 3 Model - Multilingual",
    "version": 1,
    "protocol": "aws_sigv4",
    "credential": {"roleArn": oss_execution_role},
    "parameters": {
        "region": coehre_rerank3_region,
        "service_name": "sagemaker",
        "model": coehre_rerank3_model
    },
    "actions": [
        {
            "action_type": "predict",
            "method": "POST",
            "headers": {
                "content-type": "application/json",
                "x-amz-content-sha256": "required",
            },
            "url": coehre_rerank3_endpoint,
            "request_body": '{ "documents": ${parameters.documents}, "query": "${parameters.query}", "model": "${parameters.model}", "top_n": ${parameters.top_n} }',
            "pre_process_function": "connector.pre_process.cohere.rerank",
            "post_process_function": "connector.post_process.cohere.rerank",
        }
    ],
}

response = client.transport.perform_request(
    method="POST", url="/_plugins/_ml/connectors/_create", body=body
)

coehre_rerank3_connector_id = response["connector_id"]

body = {
    "name": "cohere-rerank3-model",
    "function_name": "remote",
    "connector_id": coehre_rerank3_connector_id,
}

response = client.transport.perform_request(
    method="POST", url="/_plugins/_ml/models/_register", body=body
)

coehre_rerank3_model_id = response["model_id"]
coehre_rerank3_model_id

'liJxU5EB_plnzCpabRgV'

In [39]:
coehre_rerank3_model_id = get_model_id("cohere-rerank3-model")
coehre_rerank3_model_id

'liJxU5EB_plnzCpabRgV'

### Bedrock の Mistral Large 2 (Lambda上のLiteLLMを経由してアクセス)

LambdaのFunction URLsへのリクエストを信用リストに追加する

In [17]:
body = {
    "persistent": {
        "plugins.ml_commons.trusted_connector_endpoints_regex": [
            "^https://runtime\\.sagemaker\\..*[a-z0-9-]\\.amazonaws\\.com/.*$",
            "^https://api\\.openai\\.com/.*$",
            "^https://api\\.cohere\\.ai/.*$",
            "^https://bedrock-runtime\\..*[a-z0-9-]\\.amazonaws\\.com/.*$",
            "^https://.*\\.lambda-url\\..*\\.on\\.aws/.*$",
        ],
    }
}

response = client.transport.perform_request(
    method="PUT",
    url="/_cluster/settings",
    body=body,
)

response

{'acknowledged': True,
 'persistent': {'plugins': {'ml_commons': {'trusted_connector_endpoints_regex': ['^https://runtime\\.sagemaker\\..*[a-z0-9-]\\.amazonaws\\.com/.*$',
     '^https://api\\.openai\\.com/.*$',
     '^https://api\\.cohere\\.ai/.*$',
     '^https://bedrock-runtime\\..*[a-z0-9-]\\.amazonaws\\.com/.*$',
     '^https://.*\\.lambda-url\\..*\\.on\\.aws/.*$']}}},
 'transient': {}}

In [18]:
mistral_large2_model = "bedrock/mistral.mistral-large-2407-v1:0"
mistral_large2_region = "us-west-2"
mistral_large2_endpoint = litellm_lambda_endpoint

body = {
    "name": "OpenAI Chat Connector",
    "description": "The connector to public OpenAI model service for GPT 3.5",
    "version": 2,
    "protocol": "http",
    "parameters": {
        "endpoint": mistral_large2_endpoint,
        "model": mistral_large2_model,
        "temperature": 0,
    },
    "credential": {
        "roleArn": oss_execution_role,
        "secretArn": oss_secret_arn,
    },
    "actions": [
        {
            "action_type": "predict",
            "method": "POST",
            "url": "${parameters.endpoint}/v1/chat/completions",
            "headers": {"Authorization": "Bearer ${credential.secretArn.openai-key}"},
            "request_body": """{ "model": "${parameters.model}", "messages": ${parameters.messages}, "temperature": ${parameters.temperature} }""",
        }
    ],
}

response = client.transport.perform_request(
    method="POST", url="/_plugins/_ml/connectors/_create", body=body
)

mistral_large2_connector_id = response["connector_id"]

body = {
    "name": "mistral-large2-model",
    "function_name": "remote",
    "connector_id": mistral_large2_connector_id,
}

response = client.transport.perform_request(
    method="POST", url="/_plugins/_ml/models/_register", body=body
)

mistral_large2_model_id = response["model_id"]
mistral_large2_model_id

'OyJgU5EB_plnzCpabBTr'

In [19]:
mistral_large2_model_id = get_model_id("mistral-large2-model")
mistral_large2_model_id

'OyJgU5EB_plnzCpabBTr'

Mistral Large 2への疎通確認

In [20]:
body = {"parameters": {"messages": [{"role": "user", "content": "which llm are you?"}]}}

response = client.transport.perform_request(
    method="POST",
    url=f"/_plugins/_ml/models/{mistral_large2_model_id}/_predict",
    body=body,
)

response

{'inference_results': [{'output': [{'name': 'response',
     'dataAsMap': {'id': 'chatcmpl-f1e0962d-f05e-454c-b79b-9f48e5f33268',
      'choices': [{'finish_reason': 'stop',
        'index': 0.0,
        'message': {'content': "I am a Large Language Model (LLM) developed by Mistral AI. I'm designed to understand and generate human-like text based on the input I receive. I can assist with a wide range of tasks, from answering questions and providing explanations to generating creative content. How can I assist you today?",
         'role': 'assistant',
         'tool_calls': None,
         'function_call': None}}],
      'created': 1723680732.0,
      'model': 'mistral.mistral-large-2407-v1:0',
      'object': 'chat.completion',
      'system_fingerprint': None,
      'usage': {'completion_tokens': 65.0,
       'prompt_tokens': 9.0,
       'total_tokens': 74.0}}}],
   'status_code': 200}]}

---

## Pipelineを作成

### Ingest pipelineを作成

In [21]:
client.ingest.put_pipeline(
    id="text-chunking-embedding-ingest-pipeline",
    body={
        "description": "A text chunking and embedding ingest pipeline",
        "processors": [
            {
                "text_chunking": {
                    "algorithm": {
                        "fixed_token_length": {
                            "token_limit": 500,
                            "overlap_rate": 0.2,
                            "tokenizer": "standard",
                        }
                    },
                    "field_map": {"body": "body_chunk"},
                }
            },
            {
                "text_embedding": {
                    "model_id": titan_embed_model_id,
                    "field_map": {"body_chunk": "body_chunk_embedding"},
                }
            },
        ],
    },
)

{'acknowledged': True}

### Search pipelineを作成

In [40]:
client.search_pipeline.put(
    id="hybrid-rerank-search-pipeline",
    body={
        "description": "Post processor for hybrid rerank search",
        "phase_results_processors": [
            {
                "normalization-processor": {
                    "normalization": {"technique": "min_max"},
                    "combination": {
                        "technique": "arithmetic_mean",
                        "parameters": {"weights": [0.3, 0.7]},
                    },
                }
            }
        ],
        "response_processors": [
            {
                "rerank": {
                    "ml_opensearch": {"model_id": coehre_rerank3_model_id},
                    "context": {
                        "document_fields": ["body_chunk"],
                    },
                }
            },
            {
                "retrieval_augmented_generation": {
                    "model_id": mistral_large2_model_id,
                    "context_field_list": ["body"],
                    "system_prompt": "You are a helpful assistant. You must answer in Japanese.",
                    "user_instructions": "Generate a concise and informative answer in less than 100 words for the given question. You must answer in Japanese.",
                }
            },
        ],
    },
)

{'acknowledged': True}

## インデックスを作成

In [23]:
index_name = "rag_index"

In [24]:
client.indices.create(
    index=index_name,
    body={
        "settings": {
            "default_pipeline": "text-chunking-embedding-ingest-pipeline",
            "index": {
                "analysis": {
                    "analyzer": {
                        "default": {
                            "type": "custom",
                            "tokenizer": "sudachi_tokenizer",
                            "filter": [
                                "sudachi_baseform",
                                "sudachi_part_of_speech",
                                "sudachi_ja_stop",
                            ],
                        },
                        "default_search": {
                            "type": "custom",
                            "tokenizer": "sudachi_tokenizer",
                            "filter": [
                                "sudachi_baseform",
                                "sudachi_part_of_speech",
                                "sudachi_ja_stop",
                            ],
                        },
                    }
                },
                "knn": True,
            },
        },
        "mappings": {
            "properties": {
                "body": {"type": "keyword"},
                "body_chunk": {
                    "type": "text",
                },
                "body_chunk_embedding": {
                    "type": "nested",
                    "properties": {"knn": {"type": "knn_vector", "dimension": 1024}},
                },
            }
        },
    },
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'rag_index'}

---

## データ登録

In [25]:
from pathlib import Path
import json

directory = Path("about-aws")

for file_path in directory.glob("**/*"):
    if file_path.is_file():
        with file_path.open() as f:
            document = json.load(f)

            client.index(index=index_name, body=document)

## データ検索

In [26]:
response = client.search(
    index=index_name,
    body={
        "query": {"match_all": {}},
        "size": 1,
    },
)

print(json.dumps(response, indent=2, ensure_ascii=False))

{
  "took": 1232,
  "timed_out": false,
  "_shards": {
    "total": 5,
    "successful": 5,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1105,
      "relation": "eq"
    },
    "max_score": 1.0,
    "hits": [
      {
        "_index": "rag_index",
        "_id": "VCJhU5EB_plnzCpaGBQp",
        "_score": 1.0,
        "_source": {
          "body_chunk": [
            "本日、Research and Engineering Studio (RES) on AWS バージョン 2024.04 のリリースを発表いたします。今回の最新リリースでは、RES 仮想デスクトップに関する新しいカスタマイズオプションと、共有ストレージと仮想デスクトップインフラストラクチャ (VDI) のストリーミングに関する新しいオプションが追加されました。  \n\n\nRES on AWS バージョン 2024.04 のユーザーは、カスタムの Amazon マシンイメージ (AMI) を使用して VDI を起動できます。依存関係をカスタム AMI に事前に変換すると、VDI インスタンスの起動時間が短縮されるだけでなく、顧客がパブリックインターネットにアクセスせずに独立したプライベートサブネット内で RES VDI を起動できるようになります。顧客はインスタンスプロファイル、セキュリティグループ、起動スクリプトなどのカスタム起動パラメータを指定して、VDI をさらにカスタマイズできます。  \n\n\n今回のリリースには、共有ストレージに関する別のオプションを RES ユーザーに提供する Amazon FSx for Lustre の追加サポート、高レイテンシー環境でのストリーミングを改善する Quick UDP Internet Connections (QU

## ハイブリッドリランク検索

In [36]:
def search(query: str):
    response = client.search(
        index=index_name,
        body={
            "_source": {"exclude": ["body_chunk_embedding"]},
            "query": {
                "hybrid": {
                    "queries": [
                        {
                            "match": {
                                "body_chunk": {
                                    "query": query,
                                }
                            }
                        },
                        {
                            "nested": {
                                "score_mode": "max",
                                "path": "body_chunk_embedding",
                                "query": {
                                    "neural": {
                                        "body_chunk_embedding.knn": {
                                            "query_text": query,
                                            "model_id": titan_embed_model_id,
                                        }
                                    }
                                },
                            }
                        },
                    ],
                }
            },
            "size": 4,
            "ext": {
                "rerank": {
                    "query_context": {
                        "query_text": query,
                    },
                },
                "generative_qa_parameters": {
                    "llm_model": "litellm",
                    "llm_question": query,
                    "context_size": 4,
                },
            },
        },
        params={"search_pipeline": "hybrid-rerank-search-pipeline"},
    )

    context = list(map(lambda x: x["_source"], response["hits"]["hits"]))

    for tmp in context:
        del tmp["body_chunk"]

    return {
        "answer": response["ext"]["retrieval_augmented_generation"]["answer"],
        "context": context,
    }

---

## 回答生成

### テストデータの読み込み

In [28]:
df = pd.read_json("testdataset.json")

question = df["question"]

1件実行

In [37]:
response = search(query=question[0])
response

{'answer': 'はい、Amazon SageMaker StudioのJupyterLab Notebookでデータを探索し、変換することができます。SageMaker Studioは、SQLとPythonを組み合わせてデータを効率的に処理し、分析するための強力なツールを提供しています。これにより、データサイエンティストはデータの視覚化、探索、変換を簡単に行うことができます。',
 'context': [{'title': 'Amazon SageMaker Studio ノートブックが、インタラクティブなデータ探索と SQL クエリの実行のサポートを開始',
   'body': 'Amazon SageMaker Studio の JupyterLab Notebook には、SQL 拡張機能が組み込まれています。これにより、データサイエンティストは、ノートブックから SQL と Python を使用して複数のデータソースのデータをシームレスに発見、探索、変換できます。\n\nStudio ノートブックを扱うデータサイエンティストは、AWS Glue 接続を介して Amazon Athena、Amazon Redshift、Snowflake などの一般的なデータサービスにシームレスに接続できるようになりました。管理者はこれらの接続を安全に管理できるため、データサイエンティストは認証情報を手動で管理しなくても、承認されたデータにアクセスできます。データソースに接続すると、データサイエンティストはデータベース、スキーマ、テーブル、およびビューを簡単に参照および検索でき、ノートブックインターフェイス内でデータをプレビューできます。その後、SQL コードと Python コードを同じノートブックに混在させて、機械学習プロジェクトで使用するデータを効率的に探索および変換できます。SQL コマンド補完、コードフォーマット支援、構文の強調表示などの追加機能により、コード開発を迅速化し、開発者全体の生産性を向上させることができます。SageMaker Studio は、一般的なデータサービス、SQL/Python でのデータ探索、エンドツーエンドの機械学習を統一されたユーザーインターフェイスに統合することで、データサイエンティストが分析や機械学習の作業中にツールを切

### 回答生成開始

In [41]:
answers = []
citations = []

for q in question:
    response = search(query=q)

    answer = response["answer"]
    citation = response["context"]

    answers.append(answer)
    citations.append(citation)

    print(q)
    print(answer)

    print("---")

データサイエンティストはAmazon SageMaker Studioノートブックでどのようにデータを探索し、変換することができますか？
はい、Amazon SageMaker StudioのJupyterLab Notebookでデータを探索し、変換することができます。SageMaker Studioは、SQLとPythonを組み合わせてデータを効率的に処理し、分析するための強力なツールを提供しています。これにより、データサイエンティストはデータの視覚化、探索、変換を簡単に行うことができます。
---
フルマネージド型のデータベースインスタンスを作成するには、どのような手順を踏む必要がありますか?
フルマネージド型のデータベースインスタンスを作成するには、以下の手順が必要です。

1. **IAM ロールの作成**: `iam:CreateRole` と `iam:AttachRolePolicy` を使用して、必要な IAM ロールを作成します。
2. **インスタンスプロファイルの作成**: `iam:CreateInstanceProfile` と `iam:AddRoleToInstanceProfile` を使用して、インスタンスプロファイルを作成し、ロールを追加します。
3. **RDS マネジメントコンソールの使用**: RDS マネジメントコンソールを使用して、新しいインスタンスプロファイルの作成を選択し、データベースの作成、データベースの作成、スナップショットの有効化、ポイントインタイムリカバリーオプションを指定します。

これらの手順を完了することで、フルマネージド型のデータベースインスタンスを作成することができます。
---
AWS CloudShell でコンテナの起動はどのように簡単になったのでしょうか?
AWS CloudShell でコンテナの起動は、Docker コンテナを使用して簡単に行うことができます。CloudShell は、AWS リソースの管理、検索、操作を簡単に行うためのブラウザベースのシェルです。Docker コンテナを起動するには、CloudShell 環境から Docker コマンドを実行し、コンテナ内でコマンドを直接実行することができます。これにより、ローカルインストールや設定を行わずに、コンテナを効率的に管理で

### 回答保存

In [42]:
df["oss_answers"] = answers
df["oss_citations"] = citations

In [43]:
df.to_json("answer_data_oss.json", force_ascii=False)

# Evaluation

In [44]:
df = pd.read_json("answer_data_oss.json")
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,oss_answers,oss_citations
0,データサイエンティストはAmazon SageMaker Studioノートブックでどのよう...,"[{\n ""title"": ""Amazon SageMaker Studio ノートブック...",データサイエンティストは、Amazon SageMaker Studio の Jupyter...,simple,[{'source': 'about-aws/whats-new/2024/04/amazo...,True,はい、Amazon SageMaker StudioのJupyterLab Notebook...,[{'title': 'Amazon SageMaker Studio ノートブックが、イン...
1,フルマネージド型のデータベースインスタンスを作成するには、どのような手順を踏む必要がありますか?,[照してください。 [Amazon RDS マネジメントコンソール](https://con...,フルマネージド型の M7g または R7g データベースインスタンスを作成するには、Amaz...,simple,[{'source': 'about-aws/whats-new/2024/02/amazo...,True,フルマネージド型のデータベースインスタンスを作成するには、以下の手順が必要です。\n\n1....,[{'title': 'ワンクリックでインスタンスプロファイルを作成して RDS Custo...
2,AWS CloudShell でコンテナの起動はどのように簡単になったのでしょうか?,"[{\n ""title"": ""AWS CloudShell は 13 のリージョンで Do...",AWS CloudShell では Docker の組み込みサポートを開始したため、デベロッ...,simple,[{'source': 'about-aws/whats-new/2024/01/aws-c...,True,AWS CloudShell でコンテナの起動は、Docker コンテナを使用して簡単に行う...,[{'title': 'AWS CloudShell は 13 のリージョンで Docker...
3,高い総スループットを実現するために、Mountpoint for Amazon S3 CSI...,"[{\n ""title"": ""Mountpoint for Amazon S3 CSI ド...",Mountpoint for Amazon S3 CSI ドライバーは、Bottlerock...,simple,[{'source': 'about-aws/whats-new/2024/03/mount...,True,Mountpoint for Amazon S3 CSI ドライバーは、Amazon Ela...,[{'title': 'Mountpoint for Amazon S3 CSI ドライバー...
4,Amazon Time Sync Serviceはどのようにして追加料金なしで使用できるのですか?,[タンスタイプにも拡大する予定です。このサービスは追加料金なしで使用できます。\n\nAma...,与えられた質問に対する答えは文脈の中に存在しない,simple,[{'source': 'about-aws/whats-new/2024/03/amazo...,True,Amazon Time Sync Serviceは、AWSのリージョンやインスタンスタイプに...,[{'title': 'Amazon Time Sync Service が、米国東部 (バ...


In [45]:
question = df["question"]
answer = df["oss_answers"]
ground_truth = df["ground_truth"]
context = []

for citations in df["oss_citations"]:
    tmp = []
    for citation in citations:
        body = json.dumps(citation, ensure_ascii=False)
        tmp.append(body)

    context.append(tmp)

In [46]:
from datasets import Dataset

eval_data = Dataset.from_dict(
    {
        "question": question,
        "answer": answer,
        "ground_truth": ground_truth,
        "contexts": context,
    }
)

In [47]:
import os

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

llm = AzureChatOpenAI(azure_deployment=os.getenv("AZURE_DEPLOYMENT_GPT4OMINI", None))
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_EMBEDDINGS", None)
)

In [48]:
from ragas.metrics import (
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness,
)
from ragas.run_config import RunConfig

from ragas import evaluate

run_config = RunConfig(max_wait=600, max_retries=100)

result = evaluate(
    eval_data,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=llm,
    embeddings=embeddings,
    run_config=run_config,
    raise_exceptions=False,
)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [49]:
result

{'context_precision': 0.9083, 'faithfulness': 0.6729, 'answer_relevancy': 0.6990, 'context_recall': 0.9000}

In [50]:
eval_df = result.to_pandas()
eval_df.head()

,question,answer,ground_truth,contexts,context_precision,faithfulness,answer_relevancy,context_recall
0,データサイエンティストはAmazon SageMaker Studioノートブックでどのよう...,はい、Amazon SageMaker StudioのJupyterLab Notebook...,データサイエンティストは、Amazon SageMaker Studio の Jupyter...,"[{""title"": ""Amazon SageMaker Studio ノートブックが、イン...",0.916667,0.428571,0.798520,1.0
1,フルマネージド型のデータベースインスタンスを作成するには、どのような手順を踏む必要がありますか?,フルマネージド型のデータベースインスタンスを作成するには、以下の手順が必要です。\n\n1....,フルマネージド型の M7g または R7g データベースインスタンスを作成するには、Amaz...,"[{""title"": ""ワンクリックでインスタンスプロファイルを作成して RDS Custo...",0.583333,0.666667,0.983010,1.0
2,AWS CloudShell でコンテナの起動はどのように簡単になったのでしょうか?,AWS CloudShell でコンテナの起動は、Docker コンテナを使用して簡単に行う...,AWS CloudShell では Docker の組み込みサポートを開始したため、デベロッ...,"[{""title"": ""AWS CloudShell は 13 のリージョンで Docker...",1.000000,0.600000,0.790604,1.0
3,高い総スループットを実現するために、Mountpoint for Amazon S3 CSI...,Mountpoint for Amazon S3 CSI ドライバーは、Amazon Ela...,Mountpoint for Amazon S3 CSI ドライバーは、Bottlerock...,"[{""title"": ""Mountpoint for Amazon S3 CSI ドライバー...",1.000000,1.000000,0.648141,1.0
4,Amazon Time Sync Serviceはどのようにして追加料金なしで使用できるのですか?,Amazon Time Sync Serviceは、AWSのリージョンやインスタンスタイプに...,与えられた質問に対する答えは文脈の中に存在しない,"[{""title"": ""Amazon Time Sync Service が、米国東部 (バ...",0.583333,0.000000,0.000000,1.0


In [51]:
eval_df.to_json("eval_data_oss.json", force_ascii=False)

# 環境削除

In [52]:
%%bash

cd opensearch-cdk/
cdk destroy --all --force

LiteLLMLambdaCdkStack: destroying... [1/2]
LiteLLMLambdaCdkStack |   0 | 12:34:39 AM | DELETE_IN_PROGRESS   | AWS::CloudFormation::Stack | LiteLLMLambdaCdkStack User Initiated
LiteLLMLambdaCdkStack |   0 | 12:34:41 AM | DELETE_IN_PROGRESS   | AWS::Lambda::Permission   | litellmFunction/invoke-function-url (litellmFunctioninvokefunctionurlB01E1558) 
LiteLLMLambdaCdkStack |   0 | 12:34:41 AM | DELETE_IN_PROGRESS   | AWS::Lambda::Url          | litellmFunction/FunctionUrl (litellmFunctionFunctionUrlA1625E01) 
LiteLLMLambdaCdkStack |   0 | 12:34:41 AM | DELETE_IN_PROGRESS   | AWS::CDK::Metadata        | CDKMetadata/Default (CDKMetadata) 
LiteLLMLambdaCdkStack |   1 | 12:34:41 AM | DELETE_COMPLETE      | AWS::CDK::Metadata        | CDKMetadata/Default (CDKMetadata) 
LiteLLMLambdaCdkStack |   2 | 12:34:41 AM | DELETE_COMPLETE      | AWS::Lambda::Permission   | litellmFunction/invoke-function-url (litellmFunctioninvokefunctionurlB01E1558) 
LiteLLMLambdaCdkStack |   3 | 12:34:42 AM | DELETE_CO